In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## 1. Import Library 

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


## 2. Load the training data

In [ ]:
# Load the training data
train_data = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
test_data = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

In [ ]:
# Separate the features (pixel values) and labels
x_train = train_data.drop('label', axis=1).values
y_train = train_data['label'].values

In [ ]:
# Check if 'label' column exists in test data
if 'label' in test_data.columns:
    x_test = test_data.drop('label', axis=1).values
    y_test = test_data['label'].values
else:
    x_test = test_data.values
    y_test = None

## 3. Normalize the data

In [ ]:
# Normalize the data
x_train, x_test = x_train / 255.0, x_test / 255.0

## 3. Reshape the data 

In [ ]:
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

## 4. Visualize some sample images

In [ ]:
# Visualize some sample images
def plot_sample_images(x, y, num_samples=10):
    plt.figure(figsize=(10, 1))
    for i in range(num_samples):
        plt.subplot(1, num_samples, i + 1)
        plt.imshow(x[i].squeeze(), cmap='gray')
        plt.title(y[i] if y is not None else "No label")
        plt.axis('off')
        plt.show()
        
        
plot_sample_images(x_train, y_train)

## 5. Build the neural network model

In [ ]:

model = models.Sequential([
    layers.Flatten(input_shape=(28, 28)),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(10, activation='softmax')
])

## 6. Compile the model

In [ ]:

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Data Augmentation
data_augmentation = tf.keras.Sequential([
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])


In [ ]:
# Apply data augmentation to training data
augmented_x_train = data_augmentation(x_train).numpy()

In [ ]:
# Early Stopping Callback
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [ ]:
# Train the model with validation split and early stopping
history = model.fit(x_train, y_train, epochs=20, validation_split=0.2, callbacks=[early_stopping])


## 7. Plot training and validation accuracy

In [ ]:

# Plot training and validation accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
# Visualize some augmented images
plot_sample_images(augmented_x_train, y_train)

## 8. Save the DataFrame to a CSV file

In [ ]:
# Make predictions on the test set
test_predictions = model.predict(x_test)
predicted_labels = np.argmax(test_predictions, axis=1)

# Create a DataFrame with the ImageId and Label columns
submission = pd.DataFrame({
    "ImageId": np.arange(1, len(predicted_labels) + 1),
    "Label": predicted_labels
})

# Save the DataFrame to a CSV file
submission.to_csv("submission.csv", index=False)

print("Predictions have been saved to submission.csv")


## 9. Model Summary:
* The model is a simple feedforward neural network built using TensorFlow and Keras. It consists of:

* Input Layer: A Flatten layer that reshapes the 28x28 pixel images into a 1D array of 784 pixels.
* Hidden Layer: A Dense layer with 128 units and ReLU activation, which introduces non-linearity and learns complex patterns.
* Dropout Layer: A Dropout layer with a rate of 0.2 to prevent overfitting by randomly setting 20% of the input units to zero during training.
* Output Layer: A Dense layer with 10 units and softmax activation, which outputs a probability distribution across the 10 digit classes.